# Strands マルチエージェントシステムと AgentCore Memory Tool（短期メモリ）- MemoryManager 使用

## はじめに

このノートブックでは、AWS AgentCore Memory と Strands フレームワークを使用して**共有メモリを持つマルチエージェントシステム**を実装する方法を紹介します。以前の例では単一エージェントのメモリに焦点を当てていましたが、このノートブックでは、複数の専門エージェントが共通のメモリストアにアクセスしながら協働する方法を探ります。

**注意: これは元の MemoryClient の代わりに MemoryManager と MemorySessionManager を使用した短期メモリサンプルバージョンです。**

## チュートリアルの詳細

| 情報               | 詳細                                                                            |
|:-------------------|:--------------------------------------------------------------------------------|
| チュートリアルタイプ | 短期会話メモリ                                                                  |
| エージェントのユースケース | 旅行計画アシスタント                                                            |
| エージェントフレームワーク | Strands Agents                                                                 |
| LLM モデル          | Anthropic Claude Haiku 4.5                                                     |
| チュートリアルコンポーネント | AgentCore 短期メモリ、Strands Agents、ツール経由のメモリ取得                      |
| 難易度              | 初級                                                                            |


学習内容：

- 複数のエージェントがアクセスできる共有メモリリソースのセットアップ方法
- 独自のメモリアクセスを持つ専門エージェントのツールとしての作成
- 専門エージェントに委任するコーディネーターエージェントの実装
- 複数のエージェントインタラクション間での会話コンテキストの維持

### シナリオのコンテキスト

この例では、以下を含む**旅行計画システム**を作成します：
1. 航空旅行に特化した Flight Booking アシスタント
2. 宿泊施設に焦点を当てた Hotel Booking アシスタント
3. これらの専門エージェントに委任する Travel コーディネーター

このアプローチは、複雑なドメインを同じメモリストアを共有する専門エージェントに分解する方法を実演します。

## アーキテクチャ
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 前提条件
- Python 3.10以上
- 適切な権限を持つ AWS アカウント
- AgentCore Memory の適切な権限を持つ AWS IAM ロール
- Amazon Bedrock モデルへのアクセス

環境のセットアップと共有メモリリソースの作成から始めましょう！

## ステップ 1: 環境のセットアップ
このノートブックを動作させるために必要なすべてのライブラリのインポートとクライアントの定義から始めましょう。

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
import os
from datetime import datetime
from botocore.exceptions import ClientError
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent

# メモリ管理モジュールをインポート
from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
from bedrock_agentcore.memory.constants import ConversationalMessage, MessageRole
from bedrock_agentcore.memory.session import MemorySession, MemorySessionManager

Amazon Bedrock モデルと AgentCore に適切な権限を持つリージョンとロールを定義します

In [ ]:
REGION = os.getenv('AWS_REGION', 'us-west-2')
MODEL_ID = "global.anthropic.claude-haiku-4-5-20251001-v1:0"

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")

## ステップ 2: 共有メモリの作成
このセクションでは、専門エージェント間で共有されるメモリリソースを作成します。

In [ ]:
memory_manager = MemoryManager(region_name=REGION)

try:
    print("メモリを作成中...")
    memory_name = "TravelAgent_STM_%s" % datetime.now().strftime("%Y%m%d%H%M%S")

    # メモリリソースを作成
    memory = memory_manager.get_or_create_memory(
        name=memory_name,
        strategies=[],  # 短期メモリにはストラテジーなし
        description="Short-term memory for travel agent",
        event_expiry_days=7,  # 短期メモリの保持期間
        memory_execution_role_arn=None,  # 短期メモリではオプション
    )

    # メモリ ID を抽出して表示
    memory_id = memory.id
    logger.info(f"✅ MemoryManager でメモリの作成/取得に成功しました:")
    logger.info(f"   メモリ ID: {memory_id}")
    logger.info(f"   メモリ名: {memory.name}")
    logger.info(f"   メモリステータス: {memory.status}")
except Exception as e:
    # 拡張されたエラーレポートでメモリ作成中のエラーを処理
    logger.error(f"❌ メモリ作成に失敗しました: {e}")
    logger.error(f"エラータイプ: {type(e).__name__}")
    import traceback
    traceback.print_exc()
    
    # エラー時のクリーンアップ - 部分的に作成されたメモリを削除
    if 'memory_id' in locals():
        try:
            logger.info(f"部分的に作成されたメモリのクリーンアップを試行中: {memory_id}")
            memory_manager.delete_memory(memory_id)
            logger.info(f"✅ メモリのクリーンアップに成功しました: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"❌ メモリのクリーンアップに失敗しました: {cleanup_error}")
    
    # 元の例外を再スロー
    raise

### マルチエージェントシステムの共有メモリについて

作成したメモリリソースは、旅行計画システムの共有ナレッジベースとして機能します。すべてのエージェントがこの共通のメモリストアに読み書きを行い、以下を可能にします：

1. **知識の一貫性**: すべてのエージェントが同じ情報で作業
2. **コンテキストの保持**: エージェントの遷移を超えて会話履歴が維持
3. **専門的なアクセス**: 各エージェントは独自の actor_id を持ちますが、session_id を共有

このアプローチにより、専門エージェントが自分のドメインに集中しながらも、完全な会話コンテキストの恩恵を受けることができます。

## ステップ 3: Session Manager の初期化

このセクションでは、セッションベースのメモリ操作のための MemorySessionManager を導入し、actor と session を管理する MemorySession を作成します

In [ ]:
# セッションメモリマネージャーを初期化
session_manager = MemorySessionManager(memory_id=memory.id, region_name=REGION)

logger.info(f"✅ Session manager initialized for memory: {memory.id}")
logger.info(f"Session manager type: {type(session_manager)}")

## ステップ 4: Memory Hook Provider の作成

このステップでは、メモリ操作を自動化するカスタム `MemoryHookProvider` クラスを定義します。フックは、エージェントの実行ライフサイクルの特定のポイントで実行される特別な関数です。作成するメモリフックには2つの主要な機能があります：

1. **メモリの取得**: ユーザーがメッセージを送信すると、自動的に関連する過去の会話を取得
2. **メモリの保存**: エージェントが応答した後に新しい会話を保存

**MemoryClient バージョンからの主な変更点：**
- MemoryClient の代わりに MemorySession を使用
- タプルの代わりに ConversationalMessage オブジェクトを使用
- create_event() の代わりに add_turns() を使用
- 型安全性のために MessageRole enum を使用

In [ ]:
class ShortTermMemoryHook(HookProvider):
    def __init__(self, memory_session: MemorySession, memory_id: str):
        self.memory_session = memory_session
        self.memory_id = memory_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """エージェント開始時に最近の会話履歴をロード"""
        try:
            # 事前設定済みのメモリセッションを使用（actor_id/session_id は不要）
            recent_turns = self.memory_session.get_last_k_turns(k=5)
            
            if recent_turns:
                # コンテキスト用に会話履歴をフォーマット
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        # EventMessage オブジェクトと dict 形式の両方を処理
                        if hasattr(message, 'role') and hasattr(message, 'content'):
                            role = message['role']
                            content = message['content']
                        else:
                            role = message.get('role', 'unknown')
                            content = message.get('content', {}).get('text', '')
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                logger.info(f"メモリからのコンテキスト: {context}")
                
                # エージェントのシステムプロンプトにコンテキストを追加
                event.agent.system_prompt += f"\n\n最近の会話履歴:\n{context}\n\nこのコンテキストに基づいて自然に会話を続けてください。"
                logger.info(f"✅ {len(recent_turns)} 件の最近の会話ターンを読み込みました")
            else:
                logger.info("以前の会話履歴が見つかりません")
                
        except Exception as e:
            logger.error(f"会話履歴の読み込みに失敗しました: {e}")
    
    def on_message_added(self, event: MessageAddedEvent):
        """MemorySession を使用してメッセージをメモリに保存"""
        messages = event.agent.messages
        try:
            if messages and len(messages) > 0 and messages[-1]["content"][0].get("text"):
                message_text = messages[-1]["content"][0]["text"]
                message_role = MessageRole.USER if messages[-1]["role"] == "user" else MessageRole.ASSISTANT
                
                # メモリセッションインスタンスを使用（actor_id/session_id は渡す必要なし）
                result = self.memory_session.add_turns(
                    messages=[ConversationalMessage(message_text, message_role)]
                )
                
                event_id = result['eventId']
                logger.info(f"✅ イベント ID でメッセージを保存しました: {event_id}, ロール: {message_role.value}")
                
        except Exception as e:
            logger.error(f"メモリ保存エラー: {e}")
            import traceback
            logger.error(f"完全なトレースバック: {traceback.format_exc()}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        # メモリフックを登録
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## ステップ 5: Strands Agents を使用したマルチエージェントアーキテクチャの作成
このセクションでは、航空券予約とホテル予約の専門エージェントを作成し、両方ともメモリリソースへのアクセスを共有するマルチエージェントシステムを作成します。

In [ ]:
# 必要なコンポーネントをインポート
from strands import Agent, tool

In [ ]:
# 各専門エージェント用に一意の actor ID を作成し、session ID は共有
FLIGHT_ACTOR_ID = f"flight-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
HOTEL_ACTOR_ID = f"hotel-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
SESSION_ID = f"travel-session-{datetime.now().strftime('%Y%m%d%H%M%S')}"

### メモリアクセスを持つ専門エージェントの作成

次に、専門エージェント用のシステムプロンプトを定義しましょう。各プロンプトには、エージェントが解析できる形式でメモリパラメータが含まれています：

In [ ]:
# ホテル予約スペシャリスト用のシステムプロンプト
HOTEL_BOOKING_PROMPT = f"""あなたはホテル予約アシスタントです。お客様がホテルを探し、予約を行い、宿泊施設やアメニティに関する質問に答えるお手伝いをします。
空室状況、料金、予約手順について、親切でフレンドリーな態度で明確な情報を提供してください。"""

# 航空券予約スペシャリスト用のシステムプロンプト
FLIGHT_BOOKING_PROMPT = f"""あなたは航空券予約アシスタントです。お客様がフライトを探し、予約を行い、航空会社、路線、旅行ポリシーに関する質問に答えるお手伝いをします。
フライトの空き状況、料金、スケジュール、予約手順について、親切でフレンドリーな態度で明確な情報を提供してください。"""

### エージェントツールの実装
コーディネーターエージェントが使用できるツールとして専門エージェントを実装しましょう：

In [ ]:
@tool
def flight_booking_assistant(query: str) -> str:
    """
    航空券予約クエリを処理して応答します。

    Args:
        query: 予約、スケジュール、航空会社、旅行ポリシーに関する航空券関連の質問

    Returns:
        詳細な航空券情報、予約オプション、または旅行アドバイス
    """
    try:
        # 予約アシスタント用のメモリセッションを作成
        memory_session = session_manager.create_memory_session(
            actor_id=FLIGHT_ACTOR_ID, 
            session_id=SESSION_ID
        )
        flight_memory_hooks = ShortTermMemoryHook(memory_session, memory_id)
        
        flight_agent = Agent(
            hooks=[flight_memory_hooks],
            model=MODEL_ID,
            system_prompt=FLIGHT_BOOKING_PROMPT,
            state={"actor_id": FLIGHT_ACTOR_ID, "session_id": SESSION_ID}
        )

        response = flight_agent(query)
        return str(response)
    except Exception as e:
        return f"航空券予約アシスタントでエラーが発生しました: {str(e)}"

@tool
def hotel_booking_assistant(query: str) -> str:
    """
    ホテル予約クエリを処理して応答します。

    Args:
        query: 宿泊施設、アメニティ、予約に関するホテル関連の質問

    Returns:
        詳細なホテル情報、予約オプション、または宿泊アドバイス
    """
    try:
        # 予約アシスタント用のメモリセッションを作成
        memory_session = session_manager.create_memory_session(
            actor_id=HOTEL_ACTOR_ID, 
            session_id=SESSION_ID
        )

        hotel_memory_hooks = ShortTermMemoryHook(memory_session, memory_id)

        hotel_booking_agent = Agent(
            hooks=[hotel_memory_hooks],
            model=MODEL_ID,
            system_prompt=HOTEL_BOOKING_PROMPT,
            state={"actor_id": HOTEL_ACTOR_ID, "session_id": SESSION_ID}
        )
        
        response = hotel_booking_agent(query)
        return str(response)
    except Exception as e:
        return f"ホテル予約アシスタントでエラーが発生しました: {str(e)}"

### コーディネーターエージェントの作成

最後に、これらの専門ツール間を調整するメインの旅行計画エージェントを作成しましょう：

In [ ]:
# コーディネーターエージェント用のシステムプロンプト
TRAVEL_AGENT_SYSTEM_PROMPT = """
あなたは専門ツール間を調整する包括的な旅行計画アシスタントです：
- 航空券関連のクエリ（予約、スケジュール、航空会社、路線）→ flight_booking_assistant ツールを使用
- ホテル関連のクエリ（宿泊施設、アメニティ、予約）→ hotel_booking_assistant ツールを使用
- 完全な旅行パッケージ → 必要に応じて両方のツールを使用して包括的な情報を提供
- 一般的な旅行アドバイスや簡単な旅行の質問 → 直接回答

各エージェントは、ユーザーが履歴データについて質問した場合に備えて独自のメモリを持っています。
複雑な旅行リクエストを処理する場合、両方のツールからの情報を調整して統一された旅行プランを作成してください。
複数のソースからの情報を提示する際は、明確に整理してください。
1ターンあたり最大2つの質問をしてください。メッセージは短く、顧客を圧倒しないようにしてください。
"""

In [ ]:
travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    model=MODEL_ID,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

#### マルチエージェントシステムの準備ができました！

## エージェントをテストしましょう

旅行計画シナリオでマルチエージェントシステムをテストしましょう：

In [ ]:
response = travel_agent("Hello, I would like to book a trip from LA to Madrid. From July 1 to August 2.")

In [ ]:
response = travel_agent("I would only like to focus on the flight at the moment. direct flimid-range, city center, pool, standard room")

## メモリ永続化のテスト

メモリシステムが正しく動作しているかテストするために、新しいトラベルエージェントのインスタンスを作成し、以前に保存された情報にアクセスできるか確認します：

In [ ]:
# 旅行エージェントの新しいインスタンスを作成
new_travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    model=MODEL_ID,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

# 以前の会話について質問
new_travel_agent("以前話した航空券について教えてもらえますか？")

## まとめ

このノートブックでは、以下を実演しました：

1. 複数のエージェント用の共有メモリリソースの作成方法
2. メモリアクセスを持つ専門エージェントのツールとしての実装方法
3. 会話コンテキストを維持しながら複数のエージェント間を調整する方法
4. 異なるエージェントインスタンス間でのメモリ永続化の方法

この共有メモリを持つマルチエージェントアーキテクチャは、専門ドメインを処理しながら一貫したユーザー体験を維持できる複雑な会話型 AI システムを構築するための強力なアプローチを提供します。

## クリーンアップ
このノートブックで使用したリソースをクリーンアップするためにメモリを削除しましょう。

In [ ]:
# MemoryManager を使用してメモリリソースを削除するにはコメントを解除
# try:
#     memory_manager.delete_memory(memory_id)
#     logger.info(f"✅ Deleted memory: {memory_id}")
# except Exception as e:
#     logger.error(f"Failed to delete memory: {e}")